In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
!pip install urllib3==1.21.1
!pip install requests2
!pip install pandas
!pip install lxml
!pip install beautifulsoup4

In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
def get_reviews_on_carros_na_web(car_model, n_times):

  base_url = "https://www.carrosnaweb.com.br/opiniaolista.asp?curpage={index}&fabricante=Fiat&modelo={model}"

  index = 1

  #Dictionary object used to store the reviews
  raw_data = {}

  while index <= n_times:

    url = base_url.replace("{index}", str(index))
    url = url.replace("{model}", car_model)

    #Open the page according URL
    driver.get(url)
    
    #Find the HTML element (table)    
    element = driver.find_element_by_xpath("//html//body//table[2]//tbody//tr//td[1]//table//tbody//tr[5]//td//table[1]")

    #Get the element on HTML format
    html_content = element.get_attribute('outerHTML')

    #Do a parser on HTML element
    soup = BeautifulSoup(html_content, 'lxml')

    #Find the table on parsed element
    table = soup.find('table')

    #Loop on all table rows
    for td in table.findChildren('td'):

      #Get the textual content on row  
      text = td.text
  
      # If the text on current row contains the word 'Fiat', means this is a review.
      if "Fiat" in text:
        #Split the text on lines 
        columns = text.splitlines()
        #Store the review on Dictionary
        raw_data[columns[9]] = [columns[11], columns[12], columns[13], columns[14], columns[15]]

    #After to loop all rows on table, increment the index and go to next page
    index += 1

  #Convert the reviews on Pandas DataFrame and return them
  return pd.DataFrame.from_dict(raw_data, orient='index')

In [ ]:
# Import Drive API and authenticate.
from google.colab import drive

def save_reviews_on_csv_file(reviews, filename):
  
  # Mount your Drive to the Colab VM.
  drive.mount('/gdrive', force_remount=True)

  path = "//gdrive//My Drive//" + filename

  # Write the DataFrame to CSV file.
  with open(path, 'w') as f: 

    print(reviews)   

    #Save into csv file
    reviews.to_csv(path)

In [ ]:
reviews = get_reviews_on_carros_na_web("TORO", 10)
save_reviews_on_csv_file(reviews,"toro_avaliacoes.csv")